In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("fetal_health.csv")

In [ ]:
df['fetal_health'] = df.fetal_health.astype(int)
df['fetal_health'] = df['fetal_health'].map({3: 0, 1:1, 2:2})

In [ ]:
df['fetal_health']

In [ ]:
df.fetal_health.unique()

In [ ]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
x_train = train.loc[:, train.columns != 'fetal_health'].to_numpy()
y_train = train['fetal_health'].to_numpy()
x_test = test.loc[:, test.columns != 'fetal_health'].to_numpy()
y_test = test['fetal_health'].to_numpy()

In [ ]:
clf = LogisticRegression(C=10, penalty='l1',
                                      solver='saga',
                                      multi_class='multinomial',
                                      max_iter=10000)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
print(classification_report(y_test, clf.predict(x_test)))

# UNBOX

In [ ]:
import unboxapi
from unboxapi.tasks import TaskType
from unboxapi.models import ModelType
client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

## Create function

In [ ]:
import numpy as np

def predict_proba(model, input_features: np.ndarray):
    return model.predict_proba(input_features)

In [ ]:
def predict_prob(input_features: np.ndarray):
    return clf.predict_proba(input_features)

In [ ]:
class_names = ["Pathological", "Normal", "Suspect"]

In [ ]:
feature_names = test.loc[:, test.columns != 'fetal_health'].columns.values.tolist()

In [ ]:
# Comment this out and uncomment the next section to load the project
project = client.create_project(
    name="Sklearn Fetal Health Project",
    description="Project for predicting fetal health"
)

# Use this for loading the project on subsequent runs
'''
project = client.load_project(
    name="Sklearn Fetal Health Project"
)
'''

dataset = project.add_dataframe(
    df=test,
    class_names=class_names,
    label_column_name='fetal_health',
    name="Fetal Health Sklearn Train - N3",
    description='this is my fetal health training demo dataset',
    task_type=TaskType.TabularClassification,
    feature_names=feature_names,
)
dataset.to_dict()

In [ ]:
model = project.add_model(
    function=predict_proba, 
    model=clf,
    model_type=ModelType.sklearn,
    task_type=TaskType.TabularClassification,
    class_names=class_names,
    name='Fetal Classifier - N3',
    description='this is my second tabular classification model',
    feature_names=feature_names,
    train_sample_df=train[:100],
    train_sample_label_column_name='fetal_health',
)
model.to_dict()